In [ ]:
import os

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [2]:
openai_api_key = os.environ.get("OPENAI_API_KEY")

In [3]:
loader = PyPDFDirectoryLoader(
    path="/Users/carsten/Documents/Science/LLM",
    glob="**/[!.]*.pdf",
    silent_errors=False,
    load_hidden=False,
    recursive=False,
    extract_images=False,
    password=None,
    mode="page",
    # images_to_text=None,
    headers=None,
    extraction_mode="plain",
    # extraction_kwargs = None,
)
docs = loader.load()
len(docs)

1772

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512, chunk_overlap=128, add_start_index=True
)

all_splits = text_splitter.split_documents(docs)

len(all_splits)

9438

In [5]:
all_splits[0]

Document(metadata={'producer': 'macOS Version 14.4.1 (Build 23E224) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': '2025-01-25T22:04:23+00:00', 'moddate': '2025-01-25T17:09:36-05:00', 'source': '/Users/carsten/Documents/Science/LLM/The Hundred-page Language Models Book.pdf', 'total_pages': 209, 'page': 1, 'page_label': '2', 'start_index': 0}, page_content='1')

In [6]:
# Initialize the text embedding model from OpenAI
text_embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [7]:
collection_name = "pdfs"

In [8]:
client = QdrantClient("http://localhost:6333", timeout=600)


In [ ]:
client.create_collection(
    collection_name,
    vectors_config={
        "vector": VectorParams(
            size=384,
            distance=Distance.COSINE,
        ),
    },
)

True

In [11]:
documents = []
for split in all_splits:
    documents.append(split.page_content)

In [12]:
documents[2]

"the journey from lin-ear algebra basics to the implementation of transformers.”  ― Florian Douetteau, Co-founder and CEO at Dataiku “Andriy's book is an incredibly concise, clear, and accessible introduc-tion to machine learning.”  ― Andre Zayarni, Co-founder and CEO at Qdrant “This is one of the most comprehensive yet concise handbooks out there for truly understanding how LLMs work under the hood.”  ― Jerry Liu, Co-founder and CEO at LlamaIndex    Featuring a foreword by Tomáš Mikolov and back cover text"

In [13]:
points = []

In [14]:
index = 1
for split in all_splits:
    # dense_embeddings = text_embedding_model.embed_query(split.page_content)
    # foo = len(dense_embeddings)
    # point = PointStruct(
    #     id=int(index),
    #     vector={"vector": dense_embeddings},
    #     payload={"_id": id},
    # )
    client.add(
        collection_name,
        documents=documents,
    )
    # client.upload_points(
    #     collection_name,
    #     points=[point],
    # )
    # index += 1

AssertionError: Collection have incompatible vector params: {'vector': VectorParams(size=1536, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None)}, expected fast-bge-small-en